In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl

mpl.use("Cairo")  # for saving SVGs that Affinity Designer can parse
import matplotlib.pyplot as plt
import seaborn as sns
import cloudpickle as pkl

import gumbi as gmb
from candas.test import QuantStudio

from sklearn.metrics import confusion_matrix
from scipy.integrate import trapz
from scipy.special import logit

import benchmarking as bench

import pathlib as pl

code_pth = pl.Path.cwd()  # for running in Jupyter
# code_pth = pl.Path(__file__)  # for running in terminal
fig_pth = code_pth.parent
data_pth = fig_pth / "data"
graph_pth = fig_pth / "graphics"
gen_pth = fig_pth / "generated"
graph_pth.mkdir(exist_ok=True)

In [2]:
fig_num = 8
plt.style.use('style.mplstyle')

%config InlineBackend.figure_format = 'retina'

panel = None
subpanel = None

def savefig(fig=None, title=None):
    fig = fig or plt.gcf()
    title = title or 'Fig_'+'_'.join([str(el) for el in [fig_num, panel, subpanel] if el is not None])
    print(f'Saving {title} in {graph_pth}:', end='')
    for ext in ['svg', 'png']:
        print(f' [.{ext}]', end='')
        fig.savefig(graph_pth / f'{title}.{ext}', dpi=600, transparent=True)
    print(' done')

# Figure Formatting

In [3]:
width = 1.725
height = 1.75
figsize = (width, height)
ticklabelsize = 6
labelsize = 6

mar_l = 0.4
mar_r = 0.05
mar_t = 0.05
mar_b = 0.375


def format_fig(
    fig, figsize=figsize, mar_l=mar_l, mar_r=mar_r, mar_t=mar_t, mar_b=mar_b, **kwargs
):
    """Adjust margins of subplots using figsize"""
    height, width = figsize
    fig.set_size_inches(figsize)

    plt.subplots_adjust(
        left=mar_l / width,
        right=1 - mar_r / width,
        top=1 - mar_t / height,
        bottom=mar_b / height,
        **kwargs
    )

    for ax in fig.get_axes():
        ax.tick_params(which="both", length=1.5, width=0.6, labelsize=8)

# Experimental Results

In [4]:
endpoints = pd.read_csv(data_pth / "JG073N Endpoints.csv").rename(
    columns={"SignalDifference": "FAM-HEX"}
)

ng1_endpoints = endpoints[
    (endpoints["IFI44L Copies"] > 0)
    & (endpoints["EMRI Copies"] > 0)
    & (endpoints["Material"] == "RNA")
    & (endpoints["ng RNA"] == 1)
]

patients_100ng = pd.read_csv(gen_pth / "JG073 Processed patient data.csv")
patients_1ng = patients_100ng.drop(columns=["FAM89A"]).assign(
    EMRI=patients_100ng.EMRI - 2, IFI44L=patients_100ng.IFI44L - 2
)

In [5]:
ng1_ds = gmb.DataSet(ng1_endpoints, outputs=["FAM-HEX"])
ng1_gp = gmb.GP(ng1_ds).fit(
    continuous_dims=["EMRI Copies", "IFI44L Copies"],
    linear_dims=["EMRI Copies", "IFI44L Copies"],
)

In [6]:
import importlib as il

il.reload(bench)

obj = bench.Objective(
    ng1_gp, ng1_ds, patients_1ng, 2, gen_pth / "JG073 Logistic Regression Scores.csv"
)
obj.set_limits(pad=0)
obj.predict()
obj.optimize_scale_shift(show=False)
obj.set_limits(pad=0.1)
obj.predict()
obj.score_patients()
obj.build_roc()

array([[0.        , 0.        ],
       [0.05      , 0.        ],
       [0.1       , 0.        ],
       [0.15      , 0.        ],
       [0.2       , 0.        ],
       [0.25      , 0.        ],
       [0.3       , 0.        ],
       [0.35      , 0.        ],
       [0.4       , 0.        ],
       [0.4       , 0.04545455],
       [0.45      , 0.04545455],
       [0.5       , 0.04545455],
       [0.5       , 0.09090909],
       [0.55      , 0.09090909],
       [0.55      , 0.13636364],
       [0.6       , 0.13636364],
       [0.65      , 0.13636364],
       [0.7       , 0.13636364],
       [0.75      , 0.13636364],
       [0.75      , 0.18181818],
       [0.8       , 0.18181818],
       [0.8       , 0.22727273],
       [0.85      , 0.22727273],
       [0.9       , 0.22727273],
       [0.9       , 0.27272727],
       [0.9       , 0.31818182],
       [0.9       , 0.36363636],
       [0.9       , 0.40909091],
       [0.9       , 0.45454545],
       [0.9       , 0.5       ],
       [0.

In [7]:
LR_roc = []
scores = obj.LR_Scores["LR_Score_m"]
dxs = obj.LR_Scores["Diagnosis"]
thresholds = np.hstack([scores.min() - 1, sorted(scores.values), scores.max() + 1])
for thresh in thresholds:
    tn, fp, fn, tp = confusion_matrix(dxs == "Viral", scores <= thresh).ravel()
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)
    LR_roc.append([tpr, fpr])

LR_auroc = trapz(np.array(LR_roc)[:, 0], np.array(LR_roc)[:, 1])

In [8]:
fig, ax = plt.subplots(1, 1, figsize=figsize)
obj.set_limits(pad=0.1)
obj.predict()
pp = gmb.ParrayPlotter(
    x=obj.IFI44L_grid, y=obj.EMRI_grid, z=(obj.sig + 3 * obj.vshift) / obj.scale
)
pp(plt.contourf, cmap=obj.cmap, norm=mpl.colors.CenteredNorm(), zorder=-10)
cbar = pp.colorbar(ax=ax)  # , ticks=[-1, -0.5, 0, 0.5, 1])
pp(plt.contour, levels=0, colors="k", norm=mpl.colors.CenteredNorm(), zorder=-5)
cbar.set_label("FAM-HEX", fontsize=8)
cbar.set_label("")
cbar.ax.tick_params(labelsize=8)
obj.plot_observations(ax=ax, s=6**2)
obj.plot_patients(ax=ax, s=4**2)
ax.set_xticks([1, 2, 3, 4, 5, 6])
format_fig(fig, mar_l=0.325, mar_r=0.075)


panel = "B"
savefig()

fig, ax = plt.subplots(1, 1, figsize=figsize)
LR_color = "#2F193B"
CAN_color = "#A02A59"
# Plot LR_roc
tpr_values, fpr_values = zip(*LR_roc)
ax.plot(fpr_values, tpr_values, color=LR_color)
ax.set_ylabel("Sensitivity")
ax.set_xlabel("1-Specificity")
ax.set_xticks([0, 0.5, 1])
ax.set_yticks([0, 0.5, 1])
format_fig(fig, mar_l=0.45)

obj.plot_roc(ax=ax, color=CAN_color)
obj.roc["m"] = np.array(obj.roc["m"])
xl, yl = ax.get_xlim(), ax.get_ylim()

xy = (np.diff(xl) * 0.95 + xl[0], np.diff(yl) * 0.25 + yl[0])
ax.annotate("AUROC", xy=xy, ha="right", color="k")

xy = (np.diff(xl) * 0.95 + xl[0], np.diff(yl) * 0.15 + yl[0])
ax.annotate(f"LR: {LR_auroc:0.3f}", xy=xy, ha="right", color=LR_color)

xy = (np.diff(xl) * 0.95 + xl[0], np.diff(yl) * 0.05 + yl[0])
ax.annotate(f'Assay: {obj.auroc("m"):0.3f}', xy=xy, ha="right", color=CAN_color)

panel = "D"
savefig()

fig, ax = plt.subplots(1, 1, figsize=figsize)
obj.plot_comparison(
    ax=ax, scatter_kws={"s": 6**2}, line_kws={"alpha": 0.25, "zorder": -1}
)
xl, yl = ax.get_xlim(), ax.get_ylim()
xy = (np.diff(xl) * 0.95 + xl[0], np.diff(yl) * 0.05 + yl[0])
ax.annotate(f"$R^2$: {obj.reg.rvalue**2:0.3f}", xy=xy, ha="right")
format_fig(fig)


panel = "C"
savefig()

Saving Fig_8_B in /home/john/projects/CAN_paper/fig_8/graphics: [.svg] [.png] done
Saving Fig_8_D in /home/john/projects/CAN_paper/fig_8/graphics: [.svg] [.png] done
Saving Fig_8_C in /home/john/projects/CAN_paper/fig_8/graphics: [.svg] [.png] done


# Logistic Regression

In [9]:
LR_rslt_file = gen_pth / "JG073 Bayesian Logistic Regression Results.pkl"

with open(LR_rslt_file, "rb") as buff:
    LR_results = pkl.load(buff)

lg10_Copies = LR_results["data"]
summary = LR_results["summary"]
per_gene_posterior_predictive = LR_results["per_gene_posterior_predictive"]
per_gene_posterior_summary_stats = LR_results["per_gene_posterior_summary_stats"]
per_gene_lines = LR_results["per_gene_lines"]

genes = list(per_gene_posterior_predictive.keys())

In [10]:
logit_max = 5
space = "log odds"
figsize = (18, 4)
n_c = 100
copies = np.linspace(3, 7, n_c)
line_color = sns.color_palette("ch:0")[-1]

ymax = logit_max
yax1_ticks = np.arange(-(ymax), (ymax) + 0.1, 2.5)  # log-odds
yax1_ticks = [-4, -2, 0, 2, 4]  # log-odds
yax2_pts = np.array(
    [0.0001, 0.001, 0.01, 0.1, 0.5, 0.9, 0.99, 0.999, 0.9999]
)  # probabilities
yax2_ticks = logit(yax2_pts)
ylim = [-ymax, ymax]
yax1_label = ""  # 'Conditional\nLog Odds Dx'
yax2_label = ""  # 'Conditional\nProbability of Dx'
hline = 0

ax2s = []

fig, axs = plt.subplots(2, 1, figsize=figsize, sharex=True, sharey=True)
for i, g in enumerate(genes):
    ax = axs[i]
    ax.axhline(hline, color="k", zorder=0)

    plt.setp(
        ax,
        # title  = g,
        yticks=yax1_ticks,
        ylim=ylim,
        xticks=np.arange(copies.min(), copies.max() + 1),
        xlim=[
            copies[0] - (copies[-1] - copies[0]) * 0.05,
            copies[-1] + (copies[-1] - copies[0]) * 0.05,
        ],
    )
    ax.set_title(g, fontname="BreveSansText-Light", fontsize=10, pad=2)

    ax.set_ylabel(yax1_label)

    # For clarity, include a second yaxis that shows the marginal probability rather than log-odds
    ax2 = ax.twinx()
    ax2s.append(ax2)
    plt.setp(
        ax2,
        yticks=yax2_ticks,
        yticklabels=yax2_pts,
    )
    ax2.set_ylim(ax.get_ylim())
    ax2.set_ylabel(yax2_label, rotation=-90, verticalalignment="bottom")

# Plot patient data
for i, g in enumerate(genes):
    ax = axs[i]
    x = lg10_Copies[g]
    y = (lg10_Copies.Bacterial * 2 - 1) * (ymax - 0.5)

    high_color, low_color = sns.diverging_palette(220, 20, n=2)
    ax.plot(
        x[y < 0],
        y[y < 0],
        ls="none",
        marker="o",
        mfc=low_color,
        mec="none",
        ms=4,
        alpha=0.2,
    )
    ax.plot(
        x[y > 0],
        y[y > 0],
        ls="none",
        marker="o",
        mfc=high_color,
        mec="none",
        ms=4,
        alpha=0.2,
    )

for i, g in enumerate(genes):
    post = per_gene_posterior_predictive[g]

    lim = (copies > lg10_Copies[g].min()) & (copies < lg10_Copies[g].max())

    l, m, u = [np.percentile(post, p, axis=1) for p in [2.5, 50, 97.5]]
    axs[i].fill_between(copies[lim], l[lim], u[lim], alpha=0.2, facecolor=line_color)
    axs[i].plot(copies[lim], m[lim], color=line_color)
    axs[i].plot(copies[lim], post[lim, :30], alpha=0.1, color=line_color)

axs[-1].set_xlabel("log$_{10}$ Copies", labelpad=0)

fig.text(
    0.04, 0.55, "Log-Odds Bacterial", ha="center", va="center", rotation="vertical"
)
fig.text(0.99, 0.55, "Probability Bacterial", ha="right", va="center", rotation=270)

# plt.tight_layout()
format_fig(fig, mar_l=0.35, mar_r=0.5, mar_t=0.15, mar_b=0.325, hspace=0.5)

panel = "A"
savefig()

Saving Fig_8_A in /home/john/projects/CAN_paper/fig_8/graphics: [.svg] [.png] done


# Clinical Results

In [11]:
width = 2.365
height = 1.75
figsize = (width, height)

In [12]:
cmax = 40

JG073Q = (
    QuantStudio(
        data_pth / "JG073Q Febrile Signature 1ng patient sample replicates.xlsx",
        "JG073Q",
    )
    .import_data()
    .format_reactions()
    .index_reactions()
    .subtract_background()
    .normalize_reactions(cmax=cmax)
    .invert_fluorophore("HEX")
)

In [13]:
JG073Q.reactions.neaten()
JG073Q.extract_endpoints(cmax=cmax)
endpoints = JG073Q.endpoints

sample_data = pd.read_excel(data_pth / "JG073_RNA_JohnG_July_22.xlsx")

In [14]:
# List of letters from A to T
A_to_T = [
    "A",
    "B",
    "C",
    "D",
    "E",
    "F",
    "G",
    "H",
    "I",
    "J",
    "K",
    "L",
    "M",
    "N",
    "O",
    "P",
    "Q",
    "R",
    "S",
    "T",
]

samples_1ng = endpoints[
    (endpoints.Target == "1ng_CAN_FAM") & (endpoints.Sample.isin(A_to_T))
].merge(sample_data, left_on="Sample", right_on="Code")

samples_1ng_ind = endpoints[
    (endpoints.Sample.isin(A_to_T)) & (endpoints.Target.isin(["IFI44L", "EMRI"]))
].merge(sample_data, left_on="Sample", right_on="Code")

IFI44L_1ng = samples_1ng_ind[samples_1ng_ind.Target == "IFI44L"][
    ["Phenotype", "Code", "CT"]
]
EMRI_1ng = samples_1ng_ind[samples_1ng_ind.Target == "EMRI"][
    ["Phenotype", "Code", "CT"]
]
CTs_1ng = pd.merge(
    IFI44L_1ng, EMRI_1ng, on=["Phenotype", "Code"], suffixes=("_IFI44L", "_EMRI")
)

## qPCR

In [15]:
fig = plt.figure()
palette = sns.diverging_palette(220, 20, n=2)

CTs = CTs_1ng.replace(
    {
        "Phenotype": {
            "Definite Bacterial": "Bacterial",
            "Definite Viral": "Viral",
        }
    }
).rename(columns={"CT_IFI44L": "IFI44L", "CT_EMRI": "EMRI", "Code": "Sample"})

CTs_mean = CTs.groupby(["Sample", "Phenotype"]).agg("mean")
CTs_sd = CTs.groupby(["Sample", "Phenotype"]).agg("std")

l_ = (CTs_mean - 1.96 * CTs_sd).reset_index()
u_ = (CTs_mean + 1.96 * CTs_sd).reset_index()
CTs_mean = CTs_mean.reset_index()

sns.scatterplot(
    data=(CTs.groupby(["Sample", "Phenotype"]).agg("mean")),
    x="IFI44L",
    y="EMRI",
    hue="Phenotype",
    palette=palette,
    s=4**2,
)
plt.legend().remove()

from matplotlib.patches import Ellipse, Rectangle

for Dx, color in zip(["Bacterial", "Viral"], palette):
    l = l_[l_.Phenotype == Dx]
    c = CTs_mean[CTs_mean.Phenotype == Dx]
    u = u_[u_.Phenotype == Dx]

    #     plt.plot(c.IFI44L.values, c.EMRI.values, color=color, ls='None', marker='o')
    ax = plt.gca()

    for sample in c.Sample.unique():
        for sdm in [1]:
            ellipse = Ellipse(
                xy=(
                    c[c.Sample == sample].IFI44L.values,
                    c[c.Sample == sample].EMRI.values,
                ),
                width=(
                    u[u.Sample == sample].IFI44L - l[l.Sample == sample].IFI44L
                ).values
                * sdm,
                height=(u[u.Sample == sample].EMRI - l[l.Sample == sample].EMRI).values
                * sdm,
                fc=color,
                alpha=0.2,
            )
            ax.add_patch(ellipse)

plt.title("Standard qPCR CTs", fontname="BreveSansText-Light", fontsize=10, pad=2)
format_fig(fig, figsize=figsize, mar_l=0.3, mar_t=0.2, mar_b=0.5)

panel = "E"
savefig()

Saving Fig_8_E in /home/john/projects/CAN_paper/fig_8/graphics: [.svg] [.png] done


In [16]:
fig = plt.figure()

samples = samples_1ng.replace(
    {
        "Phenotype": {
            "Definite Bacterial": "Bacterial",
            "Definite Viral": "Viral",
        }
    }
)

mn, mx = (
    samples.groupby(["Sample", "Phenotype"])["SignalDifference"]
    .agg("mean")
    .describe()
    .loc[["min", "max"]]
)

normalize = lambda x: 0.5 - 3.5 * ((x - mn) / (mx - mn) * 2 - 1)


samples.loc[:, "SignalDifference"] = normalize(samples.SignalDifference)

# Max-min normalize Signaldifference to -3 to 4
# mean_vals['SignalDifference'] = 0.5-3.5*((mean_vals['SignalDifference'] - mean_vals['SignalDifference'].min()) / (mean_vals['SignalDifference'].max() - mean_vals['SignalDifference'].min())*2-1)

mean_vals = (
    samples.groupby(["Sample", "Phenotype"])["SignalDifference"]
    .agg("mean")
    .reset_index()
)

std_vals = (
    samples.groupby(["Sample", "Phenotype"])["SignalDifference"]
    .agg("std")
    .reset_index()
)

bac_order = (
    mean_vals.loc[mean_vals.SignalDifference.argsort(), :]
    .query('Phenotype=="Bacterial"')
    .index.values
    #  .sort_values('SignalDifference')
    #  ['Sample']
    #  .to_list()
)

vir_order = (
    mean_vals.loc[mean_vals.SignalDifference.argsort(), :]
    .query('Phenotype=="Viral"')
    .index.values
    #  .sort_values('SignalDifference')
    #  ['Sample']
    #  .to_list()
)

sort_idx = mean_vals.SignalDifference.argsort()[::-1]
sample_order = mean_vals.loc[sort_idx, "Sample"]

sns.stripplot(
    data=mean_vals,
    x="Sample",
    y="SignalDifference",
    hue="Phenotype",  # legend=False,
    order=sample_order,
    s=2**2,
    palette=palette,
)
plt.legend().remove()

# Plot the means +/- standard deviations
lines = (
    mean_vals.loc[sort_idx, "SignalDifference"].values[:, None]
    + 1.96
    * np.array([[-1, +1]])
    * std_vals.loc[sort_idx, "SignalDifference"].values[:, None]
)
x = np.tile(np.arange(len(sample_order)), [2, 1])

is_bac = mean_vals.loc[sort_idx, "Phenotype"] == "Bacterial"
is_vir = mean_vals.loc[sort_idx, "Phenotype"] == "Viral"

for idx, color in zip([is_bac, is_vir], palette):
    plt.plot(x[:, idx], lines[idx, :].T, color=color)

ax = plt.gca()
ax.set_xticks([])
ax.set_xlabel("\nRanked Patients")
ax.set_ylabel("FAM-HEX")
ax.axhline(0, color="black", linestyle="--", linewidth=1)


plt.title("CAN Score", fontname="BreveSansText-Light", fontsize=10, pad=2)
format_fig(fig, figsize=figsize, mar_l=0.3, mar_t=0.2, mar_b=0.55)


panel = "F"
savefig()

Saving Fig_8_F in /home/john/projects/CAN_paper/fig_8/graphics: [.svg] [.png] done
